# DATASCI 315, Group Work 5: Galaxy Count Prediction with Neural Networks

In this group work assignment, we will build models to predict the number of galaxies in images.

This lab will likely involve revising models, as we want to construct a model that achieves good accuracy.

**Instructions:** During lab section, and afterward as necessary, you will collaborate in two-person teams (assigned by the GSI) to complete the problems below. The GSI will help individual teams encountering difficulty, make announcements addressing common issues, and help ensure progress for all teams. *During lab, feel free to flag down your GSI to ask questions at any point!* Upon completion, one member of the team should submit their team's work through Canvas as HTML.

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

plt.rcParams["axes.grid"] = False

In [ ]:
torch.manual_seed(42)

# Import the Data

In [ ]:
sigma = 0.25

**Instructions:**

1. Download the following files:
   - `dataset_train_0.25_norm.pt`
   - `dataset_test_0.25_norm.pt`

   From one of these locations:
   - [Google Drive](https://drive.google.com/drive/folders/1DxOqrdON-_GBGSONRZfMcACyHUz_u8Dh?usp=sharing)
   - [Canvas: dataset_train_0.25_norm.pt](https://umich.instructure.com/files/39858361/download?download_frd=1)
   - [Canvas: dataset_test_0.25_norm.pt](https://umich.instructure.com/files/39858342/download?download_frd=1)

2. Go to the `Files` tab on the left.
3. Either create a `data` directory or change the path below.
4. Upload the dataset files to the directory by clicking the `Upload` button or dragging the files to the directory.

In [ ]:
train_images, n_train = torch.load(f"data/dataset_train_{sigma}_norm.pt", weights_only=False)
test_images, n_test = torch.load(f"data/dataset_test_{sigma}_norm.pt", weights_only=False)

## Display the Data

Let's display two random images from the train and test sets along with their corresponding galaxy counts:

In [ ]:
random_index = torch.randint(0, len(train_images))
plt.imshow(train_images[random_index], cmap="gray")
plt.title(f"Number of galaxies: {n_train[random_index]}")
plt.show()

In [ ]:
random_index = torch.randint(0, len(test_images))
plt.imshow(test_images[random_index], cmap="gray")
plt.title(f"Number of galaxies: {n_test[random_index]}")
plt.show()

# Build the Model

We will predict the number of galaxies in the images using simple feedforward neural networks.

The input to the model will be the image, and the output will be the predicted number of galaxies. Since the galaxy count is a discrete variable, we will treat this as a classification problem where each count (0, 1, 2, ..., 6) is a separate class.

In [ ]:
# Image dimension (images are 50x50 pixels)
image_dim = 50
# Number of classes (galaxy counts range from 0 to 6)
num_classes = 7

## Problem 1: Define the Model Architecture

Define a neural network model architecture that you think will work best for this classification task. For now, limit yourself to `nn.Linear` layers and activation functions (no regularization like dropout).

**Requirements:**
- Start with `nn.Flatten()` to convert the 2D image to a 1D vector
- Use one or more `nn.Linear` layers with appropriate input/output dimensions
- Include activation functions (e.g., `nn.ReLU()`) between linear layers
- The final layer should output `num_classes` values (one per galaxy count)

**Hint:** The input dimension after flattening is `image_dim * image_dim = 2500`. A good starting point is 1-3 hidden layers with 128-512 neurons each.

In [ ]:
# BEGIN SOLUTION
# Two hidden layers with 512 neurons each, ReLU activations
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(image_dim * image_dim, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, num_classes),
)
# END SOLUTION

In [ ]:
# Test assertions
assert hasattr(model, "forward"), "Model must have a forward method"
assert isinstance(model, nn.Module), "Model must be an nn.Module"

# Test that model produces correct output shape
test_input = torch.randn(1, image_dim, image_dim)
test_output = model(test_input)
expected = (1, num_classes)
assert test_output.shape == expected, f"Expected {expected}, got {test_output.shape}"

print(r"All model architecture tests passed\!")

# BEGIN HIDDEN TESTS
# Verify model has at least one hidden layer
has_hidden = sum(1 for m in model.modules() if isinstance(m, nn.Linear)) >= 2
assert has_hidden, "Model should have at least one hidden layer"
# END HIDDEN TESTS

# Train the Model

We provide the following function that resets the model weights. This is useful if you want to retrain a model after changing the training hyperparameters.

In [ ]:
def reset_model_parameters(model):
    """Reset all learnable parameters in the model to their initial values."""
    for module in model.modules():
        if hasattr(module, "reset_parameters"):
            module.reset_parameters()

## Problem 2: Implement the Training Loop

Fill out the `train` function below to train your model.

**Requirements:**
1. Initialize an optimizer (e.g., `optim.Adam` or `optim.SGD`)
2. For each epoch, iterate through the training data in batches:
   - Compute the model's predictions
   - Calculate the loss using `loss_fn`
   - Backpropagate the gradients
   - Update the model parameters
   - Zero the gradients
3. After each epoch, compute and store the training and test losses

**Hint:** Remember to call `optimizer.zero_grad()` to clear gradients, `loss.backward()` to compute gradients, and `optimizer.step()` to update parameters.

In [ ]:
def train(
    model,
    train_dataloader,
    train_images,
    train_labels,
    test_images,
    test_labels,
    num_epochs=100,
    learning_rate=0.001,
):
    """Train the model and return training and test losses for each epoch."""
    # BEGIN SOLUTION
    # Use cross-entropy loss for multi-class classification
    loss_fn = nn.CrossEntropyLoss()
    reset_model_parameters(model)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    test_losses = []

    for _epoch in range(num_epochs):
        # Training loop: iterate through batches
        for batch_images, batch_labels in train_dataloader:
            predictions = model(batch_images)
            loss = loss_fn(predictions, batch_labels.long())

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Evaluate on full train and test sets after each epoch
        with torch.no_grad():
            train_predictions = model(train_images)
            train_losses.append(loss_fn(train_predictions, train_labels).item())

            test_predictions = model(test_images)
            test_losses.append(loss_fn(test_predictions, test_labels).item())

    return train_losses, test_losses
    # END SOLUTION

In [ ]:
# Test assertions
assert callable(train), "train must be a callable function"
print(r"Training function defined successfully\!")

# BEGIN HIDDEN TESTS
assert True  # Placeholder hidden test
# END HIDDEN TESTS

Next, pick a learning rate and batch size, then train the model.

In [ ]:
# BEGIN SOLUTION
learning_rate = 1e-3
batch_size = 5000
# END SOLUTION

In [ ]:
train_dataset = TensorDataset(train_images, n_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_images, n_test)

**Note:** You can also adjust the number of epochs if you think the model needs more training.

In [ ]:
train_losses, test_losses = train(
    model,
    train_dataloader,
    train_images,
    n_train,
    test_images,
    n_test,
    num_epochs=100,
    learning_rate=learning_rate,
)

## Visualize Training Progress

In [ ]:
plt.plot(train_losses, label="Train")
plt.plot(test_losses, label="Test")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training and Test Loss Over Time")
plt.show()

# Evaluate the Model

## Goal: Achieve Target Accuracy

**Goal:** Achieve at least 60% accuracy on the test set.

If your model is not performing well enough, go back to Problems 1 and 2 to try different model architectures and training hyperparameters, then retrain the model.

**Suggestions for improvement:**
- Adjust the number and size of hidden layers
- Try different learning rates (e.g., 1e-4, 1e-3, 1e-2)
- Experiment with batch sizes (e.g., 32, 128, 1000)
- Increase the number of training epochs

## Compute Accuracy

In [ ]:
with torch.no_grad():
    train_predictions = model(train_images)
    test_predictions = model(test_images)

In [ ]:
train_accuracy = (train_predictions.argmax(dim=1) == n_train).float().mean().item()
test_accuracy = (test_predictions.argmax(dim=1) == n_test).float().mean().item()
print(f"Train accuracy: {train_accuracy:.2f}, Test accuracy: {test_accuracy:.2f}")

## Tests

In [ ]:
# Test assertions
assert test_accuracy >= 0.60, (
    f"Test accuracy {test_accuracy:.2f} is below 60%. "
    "Try adjusting your model architecture or hyperparameters."
)
expected_shape = (len(test_images), num_classes)
assert (
    test_predictions.shape == expected_shape
), rf"Output shape {test_predictions.shape} \!= expected {expected_shape}"
print(r"All tests passed\!")

# BEGIN HIDDEN TESTS
assert test_accuracy >= 0.55, "Test accuracy should be at least 55%"
# END HIDDEN TESTS